In [ ]:
!pip install -U setuptools

In [ ]:
!pip install matplotlib==3.1.1
!pip install scikit-learn


In [ ]:
pip install pdpbox


In [ ]:
pip install --upgrade matplotlib


In [ ]:
from pdpbox import pdp, get_dataset, info_plots

In [ ]:
pip install shap

In [ ]:
import shap

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, GradientBoostingRegressor
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import mean_squared_error, r2_score
import math
from sklearn.metrics import classification_report

In [ ]:
df = pd.read_csv('correct_tiny_features.csv')

In [ ]:
df.columns

In [ ]:
pd.set_option('display.max_rows', None)  # Отобразить все строки
pd.set_option('display.max_columns', None)  # Отобразить все столбцы

In [ ]:
dataset = df[['emo_class','sentiment',
       'emoji_num', 'pos_bracket_num', 'neg_bracket_num', 'ellipsis_num',
       'сapitalized_word_num', 'sent_num', 'punct_per_sent_num',
       'sent_words_num', 'mean_sent_words_num', 'max_sent_words_num',
       'mean_words_len', 'max_words_num', 'repeated_words_num',
       'most_repeated_word_num', 'max_synt_tree_depth', 'min_synt_tree_depth',
       'relative_clause_num', 'first_person_num', 'other_person_num',
       'adjective_num', 'comparative_adjective_num', 'super_degree_num',
       'indicative_mood_num', 'imperative_mood_num', 'imperfect_verbs_num',
       'perfect_verbs_num', 'past_tense_verb_num', 'present_tense_verb_num',
       'future_tense_verb_num']]

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
target = 'emo_class'
SEED = 42

train_data, extra_data = train_test_split(dataset, test_size=0.2,
                                         stratify=dataset[target],
                                        random_state=SEED)

In [ ]:
target_values = ['sentiment',
       'emoji_num', 'pos_bracket_num', 'neg_bracket_num', 'ellipsis_num',
       'сapitalized_word_num', 'sent_num', 'punct_per_sent_num',
       'sent_words_num', 'mean_sent_words_num', 'max_sent_words_num',
       'mean_words_len', 'max_words_num', 'repeated_words_num',
       'most_repeated_word_num', 'max_synt_tree_depth', 'min_synt_tree_depth',
       'relative_clause_num', 'first_person_num', 'other_person_num',
       'adjective_num', 'comparative_adjective_num', 'super_degree_num',
       'indicative_mood_num', 'imperative_mood_num', 'imperfect_verbs_num',
       'perfect_verbs_num', 'past_tense_verb_num', 'present_tense_verb_num',
       'future_tense_verb_num']

def get_x_y(subset):

  y_set = subset[[target]]
  x_set = subset[target_values]
  return x_set, y_set

X_train, y_train = get_x_y(train_data)
X_test, Y_test = get_x_y(extra_data)

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import mean_squared_error, r2_score
import math
from sklearn.metrics import classification_report
sc = MinMaxScaler()
x_train = sc.fit_transform(X_train)
x_test = sc.transform(X_test)

x_train_df = pd.DataFrame(x_train, columns=target_values)
x_test_df = pd.DataFrame(x_test, columns=target_values)

In [ ]:
import xgboost

params = {
    "objective": "reg:squarederror",
    "n_estimators":1000,
    "max_depth": 10,
    "subsample": 0.5,
    "colsample_bytree": 0.9,
    "colsample_bylevel": 0.8,
    "reg_alpha": 0.002,
    'scale_pos_weight':1,
    'validate_parameters':10,
    "random_state": 42,
}

model = xgboost.XGBRFRegressor(**params).fit(x_train_df, y_train)

In [ ]:
MSE_train = mean_squared_error(y_train, model.predict(x_train_df))
MSE_test = mean_squared_error(Y_test, model.predict(x_test_df))
print(f'RMSE for train: {math.sqrt(MSE_train)}')
print(f'RMSE for test: {math.sqrt(MSE_test)}')

In [ ]:
r2_score(y_train, model.predict(x_train_df))

In [ ]:
r2_score(Y_test, model.predict(x_test_df))

In [ ]:
explainer = shap.Explainer(model)
shap_values = explainer(x_test_df)

In [ ]:
shap.plots.waterfall(shap_values[0])

In [ ]:
shap.summary_plot(shap_values, x_test_df)

In [ ]:
x_test_df['emo_pred'] = model.predict(x_test_df)

In [ ]:
def get_pdp_plot_figs(feature, target_param):

  fig, axes, summary_df = info_plots.target_plot(
      df=x_test_df,
      feature=feature, feature_name=feature, target='emo_pred'
  )

In [ ]:
for col in target_values:
       get_pdp_plot_figs(col, 'emo')